# Тех проблемы  
- Всё обучение было на kaggle, ссылка : https://www.kaggle.com/code/nikitamalcev99/malcev-hw-4
- git lfs в итоге меня придушил окончательно с момента загрузки моделей, так что я разогнал инференс на локальной машине

# Задание 1: сгенерировать датасет.

Вам предстоит обучить микро-GPT2 модель для генерации текста. Но для начала вам
нужно сделать датасет!
Датасет должен состоять из текстов вашего любого писателя. Требование к датасету:

*   Датасет должен быть одним txt-файлом.
*   Выложите датасет куда-нибудь в открытый доступ и загрузите в этот ноутбук с помощью wget
*   Датасет должен содержать по меньшей мере 500 тысяч символов. Лучше -- миллион.


Советы:
* www.lib.ru -- удобный сайт с txt текстов русских писателей.
* Моделировать поэзию чуть веселее, чем прозу.

## Про использованные датасеты:
Мои датасеты:
- input.txt - набор текстов, в большей степени за авторством Лермонтова М.Ю.
- names.txt - набор имён, использовался для дополнительной проверки адекватности
- add_reverse_target.txt - набор для обучения сложению


С HF:
- набор частей текстов из треков Славы КПСС для генерации поэзии

In [353]:
# !wget your-link
# Андрей - человек уважаемый, а потому имеет подписку на git, которая позволяет git lfs файлы тащить напрямую
# но я не Андрей, поэтому кривозубые крестьяне клонируют всю репу, чтобы решить вопрос

# Skip smudge - We'll download binary files later in a faster batch
!git lfs install --skip-smudge

# Do git clone here
!git clone https://github.com/mmmhdp/nlp_labs

# Fetch all the binary files in the new clone
!git lfs pull https://github.com/mmmhdp/nlp_labs

# Reinstate smudge
!git lfs install --force

Updated Git hooks.
Git LFS initialized.
Cloning into 'nlp_labs'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 125 (delta 48), reused 87 (delta 22), pack-reused 0
Receiving objects: 100% (125/125), 3.59 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Updated Git hooks.
Git LFS initialized.


# Прототип модели


In [354]:
!pip install datasets
!pip install --upgrade tiktoken
!pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 

In [355]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import tiktoken
from datasets import load_dataset

import random
import numpy as np

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

%pdb off

Automatic pdb calling has been turned OFF


In [356]:
def seed_all(seed_value):
    pass
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
#         torch.backends.cudnn.benchmark = True
#         torch.backends.cudnn.deterministic = True

In [357]:
def get_input_path(type_def: str, platform_prefix:str = "/kaggle/working", path_prefix:str ="/nlp_labs/HW_4_additional"  ) -> str:
    RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
    if RunningInCOLAB:
        platform_prefix = "/content"

    # path = platform_prefix + path_prefix
    
    path = "./HW_4_additional"
    
    match type_def:
        case "input":
            path += "/input.txt"
        case "names":
            path += "/names.txt"
        case "add":
            path += "/add_reverse_target.txt"
        case _:
            path = path
    return path


In [358]:
def get_batch(split, all_in:bool = False, task:str = "basic"):


    if all_in:
        data = torch.cat((train_data, val_data))
    else:
        data = train_data if split == 'train' else val_data

    match task:
        case "add":

            valid_start_indexes = [x for x in range(0, len(data) - BLOCK_SIZE, BLOCK_SIZE)] # 16 - это длина примера в датасете

            ix = torch.tensor([random.choice(valid_start_indexes) for _ in range(BATCH_SIZE)]).view(BATCH_SIZE,)

            x = torch.stack(
                [data[ i     : i + BLOCK_SIZE] for i in ix]
            )

            loss_mask = torch.where(
                    torch.arange(BLOCK_SIZE) % 16 <= 7
                )[0]

            y = []

            for i in ix:
                y_data = torch.clone(data[i+1 : i+BLOCK_SIZE+1])
                y_data[loss_mask] = -100 # default ignore index = -100
                y.append(y_data)

            y = torch.stack(y)

        case "basic":

            ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))

            x = torch.stack([data[i   : i+BLOCK_SIZE  ] for i in ix])
            y = torch.stack([data[i+1 : i+BLOCK_SIZE+1] for i in ix])

        case _:
            ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))

            x = torch.stack([data[i   : i+BLOCK_SIZE  ] for i in ix])
            y = torch.stack([data[i+1 : i+BLOCK_SIZE+1] for i in ix])

    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [359]:
def get_data_enc_dec(from_file:bool, type_def:str, enc_dec_type:str = "basic"):


    if from_file:
        INPUT_FILE_PATH = get_input_path(type_def)
        with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as f:
            text = f.read()

        if enc_dec_type == "basic":
            chars = sorted(list(set(text)))
            VOCAB_SIZE = len(chars)
            stoi = { ch:i for i,ch in enumerate(chars) }
            itos = { i:ch for i,ch in enumerate(chars) }

            encode = lambda s: [stoi[c] for c in s]
            decode = lambda l: ''.join([itos[i] for i in l])
        else:
            encoding = tiktoken.get_encoding("cl100k_base")
            VOCAB_SIZE = encoding.max_token_value + 1
            encode = encoding.encode
            decode = encoding.decode

        dataset = {"train":[{"text":text}]}

    else:
        encoding = tiktoken.get_encoding("cl100k_base")

        dataset = load_dataset("huggingartists/slava-kpss")

        VOCAB_SIZE = encoding.max_token_value + 1
        encode = encoding.encode
        decode = encoding.decode



    data = []
    for example in dataset["train"]:
        for el in encode(example["text"]):
            data.append(el)

    data = torch.tensor(data, dtype=torch.long)

    return data, encode, decode, VOCAB_SIZE

In [360]:
@torch.no_grad()
def estimate_loss():


    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split, task=TASK, all_in=ALL_IN)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


## Задание 2: оптимизировать модель
Ниже приведен код обучения микро-GPT2. Прежде, чем обучать, нужно немного оптимизировать код.  Объедините `Head` и `MultiHeadAttention` в один класс, который обрабатывает все головы параллельно, рассматривая головы как еще одну размерность. Это поможет избавиться от цикла в `MultiHeadAttention` и ускорить процесс обучения. Проверьте, что ваша имплементация идентична той, что приведена ниже.

### Попытка понять, а что куда в случае с доп. размерностью.


Значения размерностей:
$$B = 64$$ $$T = 256$$ $$H = 6$$ $$D = 64$$ $$C = H*D = 384$$


Внимание всё ещё описывается как:
$$
    Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_{k}}})V
$$


Рассмотрим, как будут преобразовываться размерности:
$$Q = (B,H,T,D)$$
$$K = (B,H,T,D)⇒ K^{T}=(B,H,D,T)$$
Тогда $QK$:
$$(B,H,T,D)\bullet(B,H,D,T)→(B,H,T,T)$$

Маскируем $QK$:
$$mask(QK) - \text{оставит размерности неизменными}$$

Далее $QK^{T}V\frac{1}{\sqrt{d_{k}}}$:
$$(B,H,T,T)\bullet(B,H,T,D)→(B,H,T,D)$$

И последними шагами приведём к изначальной размерности входных данных:
* Совмещаем последние измерения, связанные с головами, в одно:
$$ (B,H,T,D)\rightarrow flatten((B,H,T,D),-3,-2,-1) \rightarrow (B,T,H*D)$$
* Приводим размерности к $(B,T,C)$
$$(B,T,H*D)\bullet(H*D,C)\rightarrow (B,T,C)$$


In [361]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()

        self.D = head_size
        self.H = num_heads
        self.key = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        self.query = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        self.value = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.proj = nn.Linear(head_size * num_heads, N_EMBD)

        self.dropout = nn.Dropout(DROPOUT)


    def forward(self, x):

        B,T,C = x.shape
        H = self.H
        D = self.D

        k = self.key(x).view(B, H, T, D)
        q = self.query(x).view(B, H, T, D)
        v = self.value(x).view(B, H, T, D)

        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5

        wei = wei.masked_fill_(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = wei @ v
        out = out.view(B,T,H*D)

        out = self.dropout(self.proj(out))
        return x

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(DROPOUT),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, N_EMBD)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, N_EMBD)
        self.blocks = nn.Sequential(*[Block(N_EMBD, n_head=N_HEAD) for _ in range(N_LAYER)])
        self.ln_f = nn.LayerNorm(N_EMBD)
        self.lm_head = nn.Linear(N_EMBD, VOCAB_SIZE)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):

        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [362]:
from datetime import datetime
def save_model_with_timestamp(model, suffix=""):
    torch.save(model,f"{SOURCE_TYPE}_{datetime.now()}_{suffix}")


# Задание 3: обучите модель на ваших текстах
Используйте те параметры, которые указаны ниже и обучите модель на вашем датасете. Обучение в колабе может занять около часа. Если результаты будут выглядеть неубедительно, учите подольше. Если есть доступ к хорошей GPU, можете немного увеличить размер трансформера.


# Задание 4: сложение
Обучите GPT выполнять сложение двух чисел, то есть $a+b=c$. Возможно, вам будет полезно предсказать цифры числа $c$ в обратном порядке, поскольку типичный алгоритм сложения (который, как мы надеемся, GPT выучит) выполняется справа налево. Возможно, вы захотите модифицировать dataloader, чтобы он просто предоставлял случайные задачи и пропускал генерацию train и val. Возможно, вы захотите замаскировать loss на входных позициях a+b, которые просто определяют задачу, используя y=-1 в таргетах (см. CrossEntropyLoss ignore_index). Работает ли сложение? Постройте график точности сложения в зависимости от количества цифр в примере. Для этого сгенерируйте по 100 примеров с 2-мя цифрам, 3-мя и т.д.



# Задание 5: калькулятор (НЕ СДЕЛАНО)
Если ваш трасформер научился складывать (хотя бы до какого-то порядка), научите его умножать, делить и вычитать. Попробуйте обучить простой калькулятор (без скобок), в котором по одному разу может встречать каждый из знаков: *, /, +, -. Это не очень простая задача :) Возможно, вы захотите добавить в датасет chain of thoughts (https://arxiv.org/abs/2201.11903), а не только ответ.



## Конфигурация и тренировка

In [363]:
# hyperparameters
BATCH_SIZE = 32 # B
BLOCK_SIZE = 256 # T
MAX_ITERS = 2 #200_000
EVAL_INTERVAL = 1 #10_000
LEARNING_RATE = 3e-4
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 1 #1000
N_EMBD = 384 # C
N_HEAD = 6 # H = n_head = 6 => D = n_embd // n_head = 64 if n_embd = 384
           # но это соотношение выбрано для удобства, размерность головы может быть выбрана иначе
N_LAYER = 4
DROPOUT = 0.2
# ------------

In [364]:
seed_all(42)

FROM_FILE = True
ALL_IN = False
TASK = "basic" # как будет выглядеть батч - с маской(для сложения) или без(для всего остального)
SOURCE_TYPE = "input" # используемый датасет
ENCODER_DECODER_TYPE = "basic"
SAVE_MODEL = False

data, encode, decode, VOCAB_SIZE = get_data_enc_dec(
    from_file=FROM_FILE,
    type_def=SOURCE_TYPE,
    enc_dec_type=ENCODER_DECODER_TYPE
)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [365]:
model = GPTLanguageModel()
m = model.to(DEVICE)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

7.34612 M parameters


In [366]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        save_model_with_timestamp(model)

    xb, yb = get_batch(split='train', task=TASK, all_in=ALL_IN)

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    del xb
    del yb

step 0: train loss 5.3875, val loss 5.4085
step 1: train loss 5.0596, val loss 5.0717


## Проверка моделей (основная часть тренировки была проведена на kaggle)

In [367]:
# gpt2_slava = torch.load("/content/nlp_labs/HW_4_additional/trained_models/input_slava_final.pt")
# gpt2_addition_a = torch.load("/content/nlp_labs/HW_4_additional/trained_models/addition_basic_encoder.pt")
# gpt2_addition_b = torch.load("/content/nlp_labs/HW_4_additional/trained_models/addition_final_advanced_encoder.pt")

### Наивный энкодер и обычный текст

In [368]:
FROM_FILE = True
ALL_IN = False
TASK = "basic" # как будет выглядеть батч - с маской(для сложения) или без(для всего остального)
SOURCE_TYPE = "input" # используемый датасет
ENCODER_DECODER_TYPE = "basic"
SAVE_MODEL = False

data, encode, decode, VOCAB_SIZE = get_data_enc_dec(
    from_file=FROM_FILE,
    type_def=SOURCE_TYPE,
    enc_dec_type=ENCODER_DECODER_TYPE
)

gpt2 = torch.load(
    f="./HW_4_additional/trained_models/input_final_basic_encoder.pt",
    map_location=DEVICE
)

gpt2 = gpt2.to(DEVICE)
gpt2.eval()

sample = encode("Привет, как дела?")
context = torch.tensor(sample ,dtype=torch.long, device=DEVICE).view(-1,len(sample))
print(decode(gpt2.generate(context, max_new_tokens=100)[0].tolist()))

Привет, как дела?

- поллочишин по м?
- Заче Паю Нучшимпнона?
- го!
- И Чиейсистьномня.

- капололил намитакль, ню тс


### тексты Славы КПСС и encoder от openai 

In [369]:
FROM_FILE = False
ALL_IN = False
TASK = "basic" # как будет выглядеть батч - с маской(для сложения) или без(для всего остального)
SOURCE_TYPE = "input_slava" # используемый датасет
ENCODER_DECODER_TYPE = "tiktoker"
SAVE_MODEL = False

data, encode, decode, VOCAB_SIZE = get_data_enc_dec(
    from_file=FROM_FILE,
    type_def=SOURCE_TYPE,
    enc_dec_type=ENCODER_DECODER_TYPE
)

gpt2 = torch.load(
    f="./HW_4_additional/trained_models/input_slava_final.pt",
    map_location=DEVICE
)

gpt2 = gpt2.to(DEVICE)
gpt2.eval()

sample = encode("Привет, как дела?")
context = torch.tensor(sample ,dtype=torch.long, device=DEVICE).view(-1,len(sample))
print(decode(gpt2.generate(context, max_new_tokens=100)[0].tolist()))

Привет, как дела? Оли кип мне ответиан Солнить чуж, киртру, нужна хму кринка словно
Pow-pow-pow-pie-p, вуд-ки!
Антила заттят плати
Я крогда, с RE-paell it!
Pow-puziя? Олакие, и раскет клиздим!
Давалисе, еualaд


### сложение с наивным энкодером

In [370]:
FROM_FILE = True
ALL_IN = False
TASK = "add" # как будет выглядеть батч - с маской(для сложения) или без(для всего остального)
SOURCE_TYPE = "add" # используемый датасет
ENCODER_DECODER_TYPE = "basic"
SAVE_MODEL = False

data, encode, decode, VOCAB_SIZE = get_data_enc_dec(
    from_file=FROM_FILE,
    type_def=SOURCE_TYPE,
    enc_dec_type=ENCODER_DECODER_TYPE
)

gpt2 = torch.load(
    f="./HW_4_additional/trained_models/addition_basic_encoder.pt",
    map_location=DEVICE
)

gpt2 = gpt2.to(DEVICE)
gpt2.eval()

sample = encode("2  +2  =")
context = torch.tensor(sample ,dtype=torch.long, device=DEVICE).view(-1,len(sample))
print(decode(gpt2.generate(context, max_new_tokens=3)[0].tolist()))

2  +2  = +4


In [371]:
pos = 0
all_res = 0
with open("./HW_4_additional/add.txt", "r") as f:
    for line in f.readlines():
        sample = encode(line[:8])
        target = line[8:-2] # убать "|" и "\n"
        
        context = torch.tensor(sample ,dtype=torch.long, device=DEVICE).view(-1,len(sample))
        check = False
        
        for _ in range(100):
            pred_to_test= decode(gpt2.generate(context, max_new_tokens=1)[0].tolist())[8:]
            if target == pred_to_test[::-1]:
                check = True
        
        all_res += 1
        if check:
            pos += 1 
        
print(f"Accuracy: {pos/all_res}")        

Accuracy: 0.0


### сложение с эекодером с openai

In [372]:
FROM_FILE = True
ALL_IN = False
TASK = "add" # как будет выглядеть батч - с маской(для сложения) или без(для всего остального)
SOURCE_TYPE = "add" # используемый датасет
ENCODER_DECODER_TYPE = "tiktoken"
SAVE_MODEL = False

data, encode, decode, VOCAB_SIZE = get_data_enc_dec(
    from_file=FROM_FILE,
    type_def=SOURCE_TYPE,
    enc_dec_type=ENCODER_DECODER_TYPE
)

gpt2 = torch.load(
    f="./HW_4_additional/trained_models/addition_final_advanced_encoder.pt",
    map_location=DEVICE
)

gpt2 = gpt2.to(DEVICE)
gpt2.eval()

sample = encode("2  +2  =")
context = torch.tensor(sample ,dtype=torch.long, device=DEVICE).view(-1,len(sample))
print(decode(gpt2.generate(context, max_new_tokens = 3)[0].tolist()))

2  +2  =4   |



In [373]:
pos = 0
all_res = 0
with open("./HW_4_additional/add.txt", "r") as f:
    for line in f.readlines():
        sample = encode(line[:8])
        target = line[8:-2] # убать "|" и "\n"
        
        context = torch.tensor(sample ,dtype=torch.long, device=DEVICE).view(-1,len(sample))
        check = False
        
        for _ in range(100):
            pred_to_test= decode(gpt2.generate(context, max_new_tokens=1)[0].tolist())[8:]
            if target == pred_to_test[::-1]:
                check = True
        
        all_res += 1
        if check:
            pos += 1 
        
print(f"Accuracy: {pos/all_res}")        

Accuracy: 0.0
